In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

import requests
from bs4 import BeautifulSoup as bs
from bs4.element import Tag, NavigableString

project_path = os.path.abspath("..")

sys.path.append(project_path)

In [4]:
from src.scraper import (
    download_unit_icon,
    extract_block_data,
    extract_feat_values,
    extract_unit_data,
)

In [5]:
def fetch_page_content(url):
    """Fetches the HTML content of the page."""
    response = requests.get(url)
    response.raise_for_status()
    # Raises an error for bad responses (e.g., 404, 500)
    return response.text

In [6]:
url = "https://ageofempires.fandom.com/wiki/War_Wagon_(Age_of_Empires_III)"
html_content = fetch_page_content(url)
soup = bs(html_content, "html.parser")
infobox = soup.find('aside', class_='portable-infobox')

In [7]:
img_path = os.path.join(project_path, "images")
download_unit_icon(infobox, img_path)

Icono guardado como: 'war_wagon_de.png' en 'c:\Users\Jonathan Diaz\Documentos\Proyects\aoe3de_web_scraper\images'


In [8]:
statistic_labels = [
    "Cost",
    "Resistance",
    "Damage",
    "Bonus damage",
]

In [9]:
blocks = infobox.find_all("section")
block = blocks[0]
extract_block_data(block)

label:  Introduced in
value_type:  str
values:  Age of Empires III

label:  Type
value_type:  list
values:  ['Cavalry', 'Light ranged cavalry', 'Ranged cavalry', 'Gunpowder cavalry']

label:  Civilization(s)
value_type:  list
values:  ['Germans', "John Black's Mercenaries (Act II: Ice)", 'South Africa']

label:  Age
value_type:  list
values:  ['Fortress Age', 'Revolution (South Africa)']

label:  Required Home City Card
value_type:  str
values:  "Wagon Warfare" (South Africa)



{'Introduced in': 'Age of Empires III',
 'Type': ['Cavalry',
  'Light ranged cavalry',
  'Ranged cavalry',
  'Gunpowder cavalry'],
 'Civilization(s)': ['Germans',
  "John Black's Mercenaries (Act II: Ice)",
  'South Africa'],
 'Age': ['Fortress Age', 'Revolution (South Africa)'],
 'Required Home City Card': '"Wagon Warfare" (South Africa)'}

In [10]:
statistic_labels = [
    "Cost",
    "Resistance",
    "Damage",
    "Bonus damage",
]

rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[0]
label = row.find("h3").text
is_st_label = label in statistic_labels

elements = row.find("div", class_="pi-data-value")

values = extract_feat_values(elements, is_st_label)

print("label: ", label)
print("value_type: ", type(values))
print("values: ", values)

label:  Introduced in
value_type:  <class 'str'>
values:  Age of Empires III


In [134]:
block = blocks[2]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[1]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")
for feat in item.children:
    if isinstance(feat, Tag) and "image" in (feat.get("class") or []):
        continue
    
    print("=========================================================================================")
    print("=========================================================================================")
    print("Type: ", type(feat).__name__)
    print("Name Tag: ", feat.name)
    # print("Class: ", feat_class)
    print("Text: ", feat.text)
    print()
    print("Content: ", feat)

Type:  NavigableString
Name Tag:  None
Text:  20% 

Content:  20% 
Type:  NavigableString
Name Tag:  None
Text:   Hand

Content:   Hand
Type:  Tag
Name Tag:  br
Text:  

Content:  <br/>


In [176]:
def extract_item_vals(item, item_type: str):

    if item_type == "text":
        vals = item.text.strip()
        return vals
    
    if item_type == "list":
        full_text = ""
        vals = []
        for child in item.children:
            if child.name == "br":
                if full_text:
                    vals.append(full_text.strip()) 
                    full_text = ""
            else:
                full_text += child.text
        if full_text:
            vals.append(full_text.strip())
        return vals
    
    elif item_type == "dict":
        grouped_tags = []
        current_group = []

        for child in item.children:
            # Saltar etiquetas con class="image"
            if isinstance(child, Tag) and "image" in (child.get("class") or []):
                continue

            if child.name == "br":
                # Agregar grupo si tiene elementos
                if current_group:
                    grouped_tags.append(current_group)
                    current_group = []
            else:
                # Añadir texto no vacío
                text = child.text.strip()
                if text:
                    current_group.append(text)

        # Agregar el último grupo si no está vacío
        if current_group:
            grouped_tags.append(current_group)

        # Crear diccionario con la información identificada
        vals = {}
        for group in grouped_tags:
            try:
                vals[group[1]] = group[0]
            except IndexError:
                raise ValueError("Information could not be extracted in dict format. Values: ", group)
        return vals
    
    else:
        raise ValueError(f"Invalid item type: {item_type}")

In [191]:
block = blocks[5]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[3]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")

extract_item_vals(item, "text")

'Same as ranged attack'

In [117]:
# Save the HTML content of the infobox in a file for each block
# for block in infobox.find_all("section"):
#     block_title = block.find("h2").text
#     folder_path = os.path.join(project_path, "html", block_title)
#     os.makedirs(folder_path, exist_ok=True)
#     rows = block.find_all("div", class_="pi-item", recursive=False)
#     for row in rows:
#         label = row.find("h3").text
#         row_path = os.path.join(folder_path, f"{label}.html")
#         with open(row_path, "w") as f:
#             f.write(str(row))
    